# Point-in-Polygon Queries with GeoPandas and H3

In [22]:
import geopandas as gpd
from h3 import h3

The data is contained as a shapefile ASAM_events.shp which is inside a folder ASAM_data_download

In [23]:
zipfile = 'zip://ASAM_shp.zip!ASAM_data_download/ASAM_events.shp'
incidents = gpd.read_file(zipfile)
incidents

,reference,dateofocc,subreg,hostility_,victim_d,descriptio,hostilityt,hostilit_D,victim_l,victim_l_D,navarea,geometry
0,1990-9,1990-06-03,26,CUBAN GUNBOAT,BELESBAT QUEEN,A CUBAN GUNBOAT COMMANDEERED LUXURY YACHT BELE...,2,Naval Engagement,11,Vessel,IV,POINT (-75.13333 21.93333)
1,1990-10,1990-03-20,71,PIRATES,RO/RO SEA DRAGON,20 MARCH 1990. BORNEO. ...,1,Pirate Assault,3,Cargo Ship,XI,POINT (108.00000 3.00000)
2,1990-11,1990-03-20,61,PIRATES,RO/RO SUNRISE,20 MARCH 1990. SINGAPORE. ...,1,Pirate Assault,3,Cargo Ship,VIII,POINT (90.00000 -1.00000)
3,1989-16,1989-01-01,62,PEOPLES DEMOCRATIC REPUBLIC OF YEMEN,U.S. MARINERS,"RED SEA, YEMEN ...",2,Naval Engagement,13,Other,IX,POINT (42.00000 14.00000)
4,1989-17,1989-09-23,63,PIRATES,LASH STONEWALL JACKSON,Indian ocean ...,1,Pirate Assault,11,Vessel,VIII,POINT (80.30000 13.10000)
...,...,...,...,...,...,...,...,...,...,...,...,...
8061,2019-75,2019-09-23,57,None,None,On September 23rd a duty watchman on board a t...,9,Attempted Boarding,0,None,II,POINT (3.23333 6.28333)
8062,2020-91,2020-02-19,63,None,AL MARJO,"On 19 February, nine persons boarded the vesse...",9,Attempted Boarding,11,Vessel,VIII,POINT (71.70485 20.96191)
8063,2020-118,2019-06-13,62,None,None,ON 13.06.2019 AT POSITION 25:25.1N - 057:37E A...,6,Other,9,Tanker,IX,POINT (57.61667 25.41833)
8064,2020-1,2019-09-23,72,None,None,TWO FISHING BOATS SAILED TOWARDS TAMBISAN FOR ...,4,Kidnapping,4,Fishing Vessel,XI,POINT (119.12000 5.47000)


H3 library has 15 levels of resolution. [This table](https://h3geo.org/#/documentation/core-library/resolution-table) shows details of each level. We choose Level 3 which results in a grid size of approximately 100km.

The function `lat_lng_to_h3` converts a location's coordinates into an H3 id of the chosen level. We add a column called `h3` with the H3 cell id for the point at level 3.

In [24]:
h3_level = 3

def lat_lng_to_h3(row):
    return h3.geo_to_h3(row.geometry.y, row.geometry.x, h3_level)

incidents['h3'] = incidents.apply(lat_lng_to_h3, axis=1)
incidents

,reference,dateofocc,subreg,hostility_,victim_d,descriptio,hostilityt,hostilit_D,victim_l,victim_l_D,navarea,geometry,h3
0,1990-9,1990-06-03,26,CUBAN GUNBOAT,BELESBAT QUEEN,A CUBAN GUNBOAT COMMANDEERED LUXURY YACHT BELE...,2,Naval Engagement,11,Vessel,IV,POINT (-75.13333 21.93333),834c91fffffffff
1,1990-10,1990-03-20,71,PIRATES,RO/RO SEA DRAGON,20 MARCH 1990. BORNEO. ...,1,Pirate Assault,3,Cargo Ship,XI,POINT (108.00000 3.00000),836990fffffffff
2,1990-11,1990-03-20,61,PIRATES,RO/RO SUNRISE,20 MARCH 1990. SINGAPORE. ...,1,Pirate Assault,3,Cargo Ship,VIII,POINT (90.00000 -1.00000),8386b4fffffffff
3,1989-16,1989-01-01,62,PEOPLES DEMOCRATIC REPUBLIC OF YEMEN,U.S. MARINERS,"RED SEA, YEMEN ...",2,Naval Engagement,13,Other,IX,POINT (42.00000 14.00000),8352a9fffffffff
4,1989-17,1989-09-23,63,PIRATES,LASH STONEWALL JACKSON,Indian ocean ...,1,Pirate Assault,11,Vessel,VIII,POINT (80.30000 13.10000),83618efffffffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8061,2019-75,2019-09-23,57,None,None,On September 23rd a duty watchman on board a t...,9,Attempted Boarding,0,None,II,POINT (3.23333 6.28333),83589cfffffffff
8062,2020-91,2020-02-19,63,None,AL MARJO,"On 19 February, nine persons boarded the vesse...",9,Attempted Boarding,11,Vessel,VIII,POINT (71.70485 20.96191),8342dcfffffffff
8063,2020-118,2019-06-13,62,None,None,ON 13.06.2019 AT POSITION 25:25.1N - 057:37E A...,6,Other,9,Tanker,IX,POINT (57.61667 25.41833),8343a9fffffffff
8064,2020-1,2019-09-23,72,None,None,TWO FISHING BOATS SAILED TOWARDS TAMBISAN FOR ...,4,Kidnapping,4,Fishing Vessel,XI,POINT (119.12000 5.47000),836819fffffffff


Since all points that fall in a grid cell will have the same id, we can simply aggregate all rows with the same grid id to find all points that fall in the grid polygon. 

We use Panda's `groupby` function on the `h3` column and add a new column `count` to the output with the number of rows for each H3 id.

In [25]:
counts = incidents.groupby(['h3']).h3.agg('count').to_frame('count').reset_index()
counts

,h3,count
0,83182afffffffff,1
1,831864fffffffff,1
2,831865fffffffff,1
3,831873fffffffff,1
4,831942fffffffff,1
...,...,...
1422,83bc1afffffffff,1
1423,83bc1bfffffffff,1
1424,83c2c5fffffffff,1
1425,83c311fffffffff,1


To visualize the results or export it to a GIS, we need to convert the H3 cell ids to a geometry. The `h3_to_geo_boundary` function takes a H3 key and returns a list of coordinates that form the hexagonal cell. Since GeoPandas uses `shapely` library for constructing geometries, we convert the list of coordinates to a shapely `Polygon` object. Note the optional second argument to the `h3_to_geo_boundary` function which we have set to `True` which returns the coordinates in the *(x,y)* order compared to default *(lat,lon)*

In [26]:
from shapely.geometry import Polygon

def add_geometry(row):
    points = h3.h3_to_geo_boundary(row['h3'], True)
    return Polygon(points)

counts['geometry'] = counts.apply(add_geometry, axis=1)
counts

,h3,count,geometry
0,83182afffffffff,1,"POLYGON ((-8.90683921786222 52.69440035346253,..."
1,831864fffffffff,1,POLYGON ((-0.6310947579755037 50.6019111204840...
2,831865fffffffff,1,POLYGON ((0.3608556306599041 49.73574240198919...
3,831873fffffffff,1,POLYGON ((-5.874786713992012 49.23891353771817...
4,831942fffffffff,1,POLYGON ((-1.609056556450131 54.17706614785891...
...,...,...,...
1422,83bc1afffffffff,1,POLYGON ((33.22996795092578 -30.56864102701764...
1423,83bc1bfffffffff,1,POLYGON ((31.92552307069071 -30.54853232050392...
1424,83c2c5fffffffff,1,"POLYGON ((-58.0781500752715 -34.1297090007077,..."
1425,83c311fffffffff,1,POLYGON ((-60.29158709919604 -44.4045011786323...


We turn the dataframe to a GeoDataframe with the CRS EPSG:4326 and write it to a geopackage.

In [27]:
gdf = gpd.GeoDataFrame(counts, crs='EPSG:4326')
output_filename = 'gridcounts.gpkg'
gdf.to_file(driver='GPKG', filename=output_filename)